<a href="https://colab.research.google.com/github/serdarildercaglar/Adaptive-MT-LLM-Fine-tuning/blob/main/llama3_2_3B_and_fine_personas_structured_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate==0.34.2
!pip install aiofiles==23.2.1
!pip install aiohappyeyeballs==2.4.0
!pip install aiohttp==3.10.5
!pip install aiosignal==1.3.1
!pip install annotated-types==0.7.0
!pip install anyio==4.6.0
!pip install argilla==2.2.0
!pip install asttokens==2.4.1
!pip install async-timeout==4.0.3
!pip install attrs==24.2.0
!pip install bitsandbytes==0.42.0
!pip install certifi==2024.8.30
!pip install charset-normalizer==3.3.2
!pip install click==8.1.7
!pip install cloudpickle==3.0.0
!pip install contourpy==1.3.0
!pip install cycler==0.12.1
!pip install datasets==3.0.0
!pip install decorator==5.1.1
!pip install dill==0.3.8
!pip install diskcache==5.6.3
!pip install git+https://github.com/argilla-io/distilabel.git@2f8ef4d59c747f0b833470fcd6c85bdfbac6ec86
!pip install exceptiongroup==1.2.2
!pip install executing==2.1.0
!pip install fastapi==0.115.0
!pip install ffmpy==0.4.0
!pip install filelock==3.16.1
!pip install fonttools==4.53.1
!pip install frozenlist==1.4.1
!pip install fsspec==2024.6.1
!pip install gradio==4.44.0
!pip install gradio_client==1.3.0
!pip install h11==0.14.0
!pip install httpcore==1.0.5
!pip install httpx==0.27.2
!pip install huggingface-hub==0.25.0
!pip install idna==3.10
!pip install importlib_resources==6.4.5
!pip install interegular==0.3.3
!pip install ipython==8.27.0
!pip install jedi==0.19.1
!pip install Jinja2==3.1.4
!pip install joblib==1.4.2
!pip install jsonschema==4.23.0
!pip install jsonschema-specifications==2023.12.1
!pip install kiwisolver==1.4.7
!pip install lark==1.2.2
!pip install llama_cpp_python==0.2.85
!pip install llvmlite==0.43.0
!pip install markdown-it-py==3.0.0
!pip install MarkupSafe==2.1.5
!pip install matplotlib==3.9.2
!pip install matplotlib-inline==0.1.7
!pip install mdurl==0.1.2
!pip install mpmath==1.3.0
!pip install multidict==6.1.0
!pip install multiprocess==0.70.16
!pip install nest-asyncio==1.6.0
!pip install networkx==3.3
!pip install numba==0.60.0
!pip install numpy==1.26.4
!pip install orjson==3.10.7
!pip install outlines==0.0.36
!pip install packaging==24.1
!pip install pandas==2.2.3
!pip install parso==0.8.4
!pip install pexpect==4.9.0
!pip install pillow==10.4.0
!pip install portalocker==2.10.1
!pip install prompt_toolkit==3.0.47
!pip install psutil==6.0.0
!pip install ptyprocess==0.7.0
!pip install pure_eval==0.2.3
!pip install pyarrow==17.0.0
!pip install pydantic==2.9.2
!pip install pydantic_core==2.23.4
!pip install pydub==0.25.1
!pip install Pygments==2.18.0
!pip install pyparsing==3.1.4
!pip install python-dateutil==2.9.0.post0
!pip install python-multipart==0.0.10
!pip install pytz==2024.2
!pip install PyYAML==6.0.2
!pip install referencing==0.35.1
!pip install regex==2024.9.11
!pip install requests==2.32.3
!pip install rich==13.8.1
!pip install rpds-py==0.20.0
!pip install ruff==0.6.7
!pip install safetensors==0.4.5
!pip install scipy==1.14.1
!pip install semantic-version==2.10.0
!pip install shellingham==1.5.4
!pip install six==1.16.0
!pip install sniffio==1.3.1
!pip install stack-data==0.6.3
!pip install starlette==0.38.6
!pip install sympy==1.13.3
!pip install tblib==3.0.0
!pip install tokenizers==0.19.1
!pip install tomlkit==0.12.0
!pip install torch==2.4.1
!pip install tqdm==4.66.5
!pip install traitlets==5.14.3
!pip install transformers==4.44.2
!pip install typer==0.12.5
!pip install typing_extensions==4.12.2
!pip install tzdata==2024.1
!pip install universal_pathlib==0.2.5
!pip install urllib3==2.2.3
!pip install uvicorn==0.30.6
!pip install wcwidth==0.2.13
!pip install websockets==12.0
!pip install xxhash==3.5.0
!pip install yarl==1.11.1


In [ ]:
curl -o llama-3.2-3b-instruct-q8_0.gguf https://huggingface.co/hugging-quants/Llama-3.2-3B-Instruct-Q8_0-GGUF/resolve/main/llama-3.2-3b-instruct-q8_0.gguf?download=true

In [ ]:
from enum import Enum

from distilabel.llms import LlamaCppLLM
from distilabel.pipeline import Pipeline
from distilabel.steps import LoadDataFromHub
from distilabel.steps.tasks import TextGeneration
from pydantic import BaseModel, StringConstraints, conint
from typing_extensions import Annotated


class Weapon(str, Enum):
    sword = "sword"
    axe = "axe"
    mace = "mace"
    spear = "spear"
    bow = "bow"
    crossbow = "crossbow"


class Armor(str, Enum):
    leather = "leather"
    chainmail = "chainmail"
    plate = "plate"
    mithril = "mithril"


class Character(BaseModel):
    name: Annotated[str, StringConstraints(max_length=30)]
    age: conint(gt=1, lt=3000)
    armor: Armor
    weapon: Weapon


model_path = "llama-3.2-3b-instruct-q8_0.gguf"
system_prompt = "Generate RPG characters based on descriptions"

with Pipeline() as pipeline:
    llm = LlamaCppLLM(
        model_path=model_path,
        n_gpu_layers=-1,
        n_ctx=1024,
        structured_output={"format": "json", "schema": Character},
    )
    load_dataset = LoadDataFromHub(
        repo_id="argilla/FinePersonas-v0.1",
        num_examples=100,
        streaming=True,
        output_mappings={"persona": "instruction"},
    )
    text_generation = TextGeneration(system_prompt=system_prompt, llm=llm)
    load_dataset >> text_generation

if __name__ == "__main__":
    distiset = pipeline.run(
        parameters={
            text_generation.name: {
                "llm": {"generation_kwargs": {"max_new_tokens": 256*4}}
            }
        },
        use_cache=False,
    )
